**1. Upload the video file to data folder**


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
# Відкриваємо файли
cap = cv2.VideoCapture("data/Match.mp4")
template = cv2.imread("data/template.png", 0)
h, w = template.shape[:2]

In [ ]:
# Output data
output_size = (1080, 1920)
fourcc = cv2.VideoWriter.fourcc(*"mp4v")
out = cv2.VideoWriter("data/output_yolo.mp4", fourcc, 30.0, output_size)

In [ ]:
# Source points from video frame
pts_src = np.array([[210, 60], [945, 60], [1113, 1240], [114, 1240]], dtype=np.float32)

# Corresponding points on the "straightened" image
pts_dst = np.array([[0, 0], [1080, 0], [1080, 1920], [0, 1920]], dtype=np.float32)

# Homography matrix for perspective correction
H, _ = cv2.findHomography(pts_src, pts_dst)

In [ ]:
tracker = None
detect_threshold = 0.83  # For template matching
template_verify_threshold = 0.65  # For checking if tracking is still on the ball
frame_count = 0

In [ ]:
last_valid_positions = []  # Store last few valid positions to detect jumps
max_positions = 15  # Number of positions to keep in history
max_jump_distance = 50  # Maximum allowed jump distance between frames

⚠️ **To Fix:** Трекінг в цілому працює, але коли гравець різко б'є по м'ячу, то трекінг переключається на гравця і застрягає на ньому


In [ ]:
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply perspective transformation to the frame
        warped = cv2.warpPerspective(frame, H, (1080, 1920))
        gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)  # у сірий

        # Search for the ball
        if tracker is None:
            # If tracking is not active - search for the ball using template matching
            res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
            print(f"[Detection] Accuracy: {max_val:.3f}")

            # If the match value exceeds the threshold - object found
            if max_val >= detect_threshold:
                print(f"[Detection] Match: {max_val:.3f}")
                top_left = max_loc
                bbox = (top_left[0], top_left[1], w, h)

                # Initialize tracker
                tracker = cv2.legacy.TrackerCSRT_create()
                tracker.init(warped, bbox)

                bottom_right = (top_left[0] + w, top_left[1] + h)

                cv2.rectangle(warped, top_left, bottom_right, (255, 0, 0), 2)
                cv2.putText(
                    warped,
                    f"Detected {max_val:.3f}",
                    (top_left[0], top_left[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 0, 0),
                    2,
                )

                # Add position to history
                ball_center = (top_left[0] + w // 2, top_left[1] + h // 2)
                last_valid_positions.append(ball_center)
                if len(last_valid_positions) > max_positions:
                    last_valid_positions.pop(0)
        else:
            frame_count += 1
            success, bbox = tracker.update(warped)
            print(f"[Tracking] Tracker active, bbox: {bbox}")

            if success:
                x, y, bw, bh = [int(v) for v in bbox]
                current_center = (x + bw // 2, y + bh // 2)

                # Check if ball jumped too far - this indicates tracker drift
                if len(last_valid_positions) > 0:
                    last_pos = last_valid_positions[-1]
                    distance = np.sqrt(
                        (current_center[0] - last_pos[0]) ** 2
                        + (current_center[1] - last_pos[1]) ** 2
                    )
                    if distance > max_jump_distance:
                        print(
                            f"[Warning] Ball jumped too far ({distance:.1f} pixels), likely tracker drift"
                        )
                        tracker = None
                        continue

                # 1. Template verification check
                if frame_count % 20 == 0:  # Check template every 20 frames
                    roi = gray[y : y + bh, x : x + bw]
                    if roi.shape[0] >= h and roi.shape[1] >= w:
                        roi_resized = cv2.resize(roi, (w, h))
                        res = cv2.matchTemplate(roi_resized, template, cv2.TM_CCOEFF_NORMED)
                        _, match_val, _, _ = cv2.minMaxLoc(res)
                        print(f"[Tracking] Match to template: {match_val:.3f}")

                        if match_val < template_verify_threshold:
                            print("[Warning] Tracker drifted from template - resetting")
                            tracker = None
                            continue

                # 2. Circularity check
                roi = gray[
                    max(0, y) : min(gray.shape[0], y + bh),
                    max(0, x) : min(gray.shape[1], x + bw),
                ]
                if roi.size > 0:  # Make sure ROI is valid
                    # Apply thresholding to isolate the ball from background
                    _, roi_thresh = cv2.threshold(roi, 170, 255, cv2.THRESH_BINARY)

                    # Apply morphological operations to clean up the mask
                    kernel = np.ones((3, 3), np.uint8)
                    roi_thresh = cv2.morphologyEx(roi_thresh, cv2.MORPH_OPEN, kernel)
                    roi_thresh = cv2.morphologyEx(roi_thresh, cv2.MORPH_CLOSE, kernel)

                    contours, _ = cv2.findContours(
                        roi_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                    )
                    valid_contour = False

                    for cnt in contours:
                        if len(cnt) > 5:  # Need at least 5 points to fit an ellipse
                            area = cv2.contourArea(cnt)
                            if area > 30:  # Ignore tiny contours
                                perimeter = cv2.arcLength(cnt, True)
                                circularity = 4 * np.pi * area / (perimeter**2 + 1e-5)
                                print(f"[Circularity] Value: {circularity:.3f}")

                                if circularity > 0.7:  # Closer to 1 = circle
                                    valid_contour = True
                                    # Draw the circular contour
                                    cv2.drawContours(
                                        warped,
                                        [cnt + np.array([max(0, x), max(0, y)])],
                                        -1,
                                        (0, 255, 0),
                                        2,
                                    )

                    if (
                        not valid_contour and frame_count % 5 == 0
                    ):  # Only check every 5 frames to avoid flickering
                        print("[Warning] No circular object in ROI - might have drifted")
                        # We'll let template matching confirm this in next verification

                # If we've passed all checks, update history
                last_valid_positions.append(current_center)
                if len(last_valid_positions) > max_positions:
                    last_valid_positions.pop(0)

                # Draw rectangle around the tracked ball
                cv2.rectangle(warped, (x, y), (x + bw, y + bh), (255, 0, 0), 2)
                cv2.putText(
                    warped,
                    f"Tracked",
                    (x + 10, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 0, 0),
                    2,
                )

                # Draw trajectory
                if len(last_valid_positions) > 1:
                    for i in range(1, len(last_valid_positions)):
                        cv2.line(
                            warped,
                            last_valid_positions[i - 1],
                            last_valid_positions[i],
                            (0, 0, 255),
                            2,
                        )
            else:
                print("[Info] Tracker lost - retrying.")
                tracker = None

        # Записуємо оброблений кадр у відео
        out.write(warped)

        # зменшую розмір кадру в 2 рази для зручного перегляду
        scaled_warped = cv2.resize(warped, (0, 0), fx=0.5, fy=0.5)
        cv2.imshow("Warped View", scaled_warped)

        # вихід по клавіші Esc
        if cv2.waitKey(30) & 0xFF == 27:
            break
finally:
    cap.release()
    out.release()
    cv2.destroyAllWindows()

## Gates tracking

In [ ]:
import time
from pathlib import Path
from dataclasses import dataclass
from typing import Final, Optional, Tuple, List

import cv2
import numpy as np

@dataclass(frozen=True)
class PitchConfig:
    green_range: Tuple[Tuple[int, int, int], Tuple[int, int, int]] = ((35, 40, 40), (85, 255, 255))
    redetection_interval: int = 30        # Every N frames, re-detect the pitch
    max_corner_displacement: float = 6.0    # Maximum allowed drift for optical flow

@dataclass(frozen=True)
class GoalPostConfig:
    strip_height: int = 80
    bright_hsv_range: Tuple[Tuple[int, int, int], Tuple[int, int, int]] = ((0, 0, 200), (180, 60, 255))
    sobel_blur_kernel: int = 21
    min_peak_intensity_ratio: float = 0.30
    min_peak_gap: int = 80
    zone_fraction: float = 0.4
    max_allowed_jump: int = 30
    recheck_radius: int = 3
    recheck_threshold: float = 0.40
    smoothing_alpha: float = 0.35

LABEL_FONT: Final = cv2.FONT_HERSHEY_SIMPLEX
PITCH_CONFIG = PitchConfig()
GOAL_CONFIG = GoalPostConfig()

def order_corners(points: np.ndarray) -> np.ndarray:
    """
    Orders the given points in consistent order: top-left, top-right, bottom-right, bottom-left.
    """
    sum_vals = points.sum(axis=1)
    diff_vals = np.diff(points, axis=1)
    ordered = np.array([
        points[np.argmin(sum_vals)],   # Top-left has the smallest sum
        points[np.argmin(diff_vals)],  # Top-right has the smallest difference
        points[np.argmax(sum_vals)],   # Bottom-right has the largest sum
        points[np.argmax(diff_vals)]   # Bottom-left has the largest difference
    ], dtype=np.float32)
    return ordered

def detect_pitch(hsv_frame: np.ndarray) -> Optional[np.ndarray]:
    """
    Detects the green pitch (field) in the given HSV frame.
    Returns a 4x2 array of corner points (float32) in order: TL, TR, BR, BL.
    Returns None if no valid pitch is found.
    """
    lower_green, upper_green = PITCH_CONFIG.green_range
    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    largest_contour = max(contours, key=cv2.contourArea)
    if cv2.contourArea(largest_contour) < 50_000:
        return None

    hull = cv2.convexHull(largest_contour)
    approx_epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx_poly = cv2.approxPolyDP(hull, approx_epsilon, True)
    if len(approx_poly) != 4:
        return None

    return order_corners(approx_poly.reshape(-1, 2))

def get_bright_mask(hsv_frame: np.ndarray) -> np.ndarray:
    """
    Returns a binary mask of pixels within the bright HSV range.
    """
    lower_bright, upper_bright = GOAL_CONFIG.bright_hsv_range
    return cv2.inRange(hsv_frame, lower_bright, upper_bright)

def detect_two_peaks(color_strip: np.ndarray) -> Optional[Tuple[int, int]]:
    """
    Analyzes a vertical strip (in BGR) to detect two bright peaks corresponding to goal posts.
    Returns the x positions of the left and right peaks if they pass threshold criteria.
    """
    hsv_strip = cv2.cvtColor(color_strip, cv2.COLOR_BGR2HSV)
    mask = cv2.morphologyEx(
        get_bright_mask(hsv_strip),
        cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    )
    sobel_energy = np.abs(cv2.Sobel(mask, cv2.CV_32F, 0, 1, ksize=3)).sum(axis=0)
    sobel_energy = cv2.blur(sobel_energy[:, None], (GOAL_CONFIG.sobel_blur_kernel, 1)).ravel()
    if sobel_energy.max() < 1:
        return None

    total_width = sobel_energy.size
    left_zone = int(GOAL_CONFIG.zone_fraction * total_width)
    right_zone = int((1 - GOAL_CONFIG.zone_fraction) * total_width)
    left_peak = np.argmax(sobel_energy[:left_zone])
    right_peak = np.argmax(sobel_energy[right_zone:]) + right_zone

    threshold = GOAL_CONFIG.min_peak_intensity_ratio * sobel_energy.max()
    if (sobel_energy[left_peak] < threshold or 
        sobel_energy[right_peak] < threshold or
        (right_peak - left_peak) < GOAL_CONFIG.min_peak_gap):
        return None

    return int(left_peak), int(right_peak)

def is_bright_confirmed(hsv_strip: np.ndarray, x_position: int) -> bool:
    """
    Confirms that the vertical slice near x_position in the strip meets the required brightness fraction.
    """
    width = hsv_strip.shape[1]
    x_start = max(0, x_position - GOAL_CONFIG.recheck_radius)
    x_end = min(width, x_position + GOAL_CONFIG.recheck_radius + 1)
    brightness_fraction = (get_bright_mask(hsv_strip[:, x_start:x_end]) > 0).mean()
    return brightness_fraction >= GOAL_CONFIG.recheck_threshold

class PostTracker:
    """
    Exponential moving average tracker to stabilize detected goal posts frame-to-frame.
    """
    def __init__(self) -> None:
        self.last_confirmed_posts: Optional[Tuple[int, int]] = None

    def update(self, current_measurement: Optional[Tuple[int, int]], hsv_strip: np.ndarray) -> Optional[Tuple[int, int]]:
        """
        Updates the tracker with the current measurement if it passes confirmation tests.
        """
        if current_measurement:
            left_peak, right_peak = current_measurement
            if not (is_bright_confirmed(hsv_strip, left_peak) and is_bright_confirmed(hsv_strip, right_peak)):
                current_measurement = None

        if self.last_confirmed_posts and current_measurement:
            if max(abs(current_measurement[0] - self.last_confirmed_posts[0]),
                   abs(current_measurement[1] - self.last_confirmed_posts[1])) > GOAL_CONFIG.max_allowed_jump:
                current_measurement = None

        if current_measurement:
            if self.last_confirmed_posts:
                smoothed_left = int(GOAL_CONFIG.smoothing_alpha * current_measurement[0] +
                                    (1 - GOAL_CONFIG.smoothing_alpha) * self.last_confirmed_posts[0])
                smoothed_right = int(GOAL_CONFIG.smoothing_alpha * current_measurement[1] +
                                     (1 - GOAL_CONFIG.smoothing_alpha) * self.last_confirmed_posts[1])
                self.last_confirmed_posts = (smoothed_left, smoothed_right)
            else:
                self.last_confirmed_posts = current_measurement
        return self.last_confirmed_posts

def draw_black_goal(frame: np.ndarray) -> None:
    """
    Detects a fixed black goal in the frame and draws a rectangle if a candidate is found.
    """
    height, width = frame.shape[:2]
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    black_mask = cv2.inRange(hsv_frame, (0, 0, 0), (180, 255, 40))
    black_mask = cv2.morphologyEx(black_mask, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8), iterations=2)
    contours, _ = cv2.findContours(black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        contour_area = cv2.contourArea(cnt)
        x, y, bw, bh = cv2.boundingRect(cnt)
        if not ((width * 0.05) * (height * 0.015) < contour_area < (width * 0.15) * (height * 0.045)):
            continue
        aspect_ratio = bw / bh
        if not (2.5 < aspect_ratio < 6.0) or not (y + bh / 2 < height * 0.25) or not (width * 0.3 < x + bw / 2 < width * 0.7):
            continue
        cv2.rectangle(frame, (x, y), (x + bw, y + bh), (255, 0, 255), 2)
        break

def overlay_pitch_info(frame: np.ndarray, corners: np.ndarray) -> None:
    """
    Overlays the detected pitch polygon and its labeled corners on the frame.
    """
    polygon = corners.astype(int).reshape(-1, 2)
    cv2.polylines(frame, [polygon], True, (0, 255, 0), 2)
    corner_labels = ("TL", "TR", "BR", "BL")
    for label, (x, y) in zip(corner_labels, polygon):
        cv2.putText(frame, label, (x + 5, y - 5), LABEL_FONT, 0.5, (0, 255, 0), 1)

# -----------------------------------------------------------------------------
# Video Processing Helpers
# -----------------------------------------------------------------------------

def update_pitch_corners(
    frame: np.ndarray,
    current_corners: np.ndarray,
    prev_gray: np.ndarray,
    frame_index: int,
    lk_params: dict
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Updates the pitch corner positions either by re-detection or by optical flow tracking.
    Returns the updated pitch corners and the current grayscale frame.
    """
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if frame_index % PITCH_CONFIG.redetection_interval == 0:
        new_corners = detect_pitch(hsv_frame)
        if new_corners is not None:
            current_corners = new_corners.reshape(-1, 1, 2).astype(np.float32)

    current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    new_corners, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, current_gray, current_corners, None, **lk_params)
    if status.sum() == 4 and np.max(np.linalg.norm(new_corners - current_corners, axis=2)) < PITCH_CONFIG.max_corner_displacement:
        current_corners = new_corners

    return current_corners, current_gray

def detect_goal_in_frame(
    frame: np.ndarray, 
    pitch_corners: np.ndarray, 
    tracker: PostTracker
) -> Optional[Tuple[Tuple[int, int], Tuple[int, int]]]:
    """
    Using the lower part of the pitch (the strip near the bottom edge),
    detects goal posts and returns the goal rectangle coordinates.
    """
    polygon = pitch_corners.astype(int).reshape(-1, 2)
    x_left, x_right = polygon[:, 0].min(), polygon[:, 0].max()
    y_top, y_bottom = polygon[:, 1].min(), polygon[:, 1].max()

    strip_top = max(int(y_bottom - GOAL_CONFIG.strip_height), 0)
    strip_bottom = int(y_bottom)
    color_strip = frame[strip_top:strip_bottom, x_left:x_right]
    strip_hsv = cv2.cvtColor(color_strip, cv2.COLOR_BGR2HSV)

    measurement = detect_two_peaks(color_strip)
    post_positions = tracker.update(measurement, strip_hsv)

    if post_positions:
        left_post, right_post = post_positions
        goal_top_left = (x_left + left_post, strip_top)
        goal_bottom_right = (x_left + right_post, y_bottom)
        cv2.rectangle(frame, goal_top_left, goal_bottom_right, (255, 0, 255), 2)
        return goal_top_left, goal_bottom_right
    return None

def process_video(source_path: Path, destination_path: Optional[Path] = None) -> List[Tuple[int, Optional[Tuple[Tuple[int, int], Tuple[int, int]]]]]:
    """
    Processes the video at 'source_path', optionally saves an annotated copy to 'destination_path',
    and returns a list with each frame's index along with detected goal coordinates.
    """
    cap = cv2.VideoCapture(str(source_path))
    ret, initial_frame = cap.read()
    if not ret:
        raise RuntimeError("Could not read the initial frame from video.")
    height, width = initial_frame.shape[:2]

    cv2.namedWindow("goals", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("goals", 960, 540)
    writer = None
    if destination_path:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(str(destination_path), fourcc, 30, (width, height))

    initial_hsv = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2HSV)
    detected_corners = detect_pitch(initial_hsv)
    if detected_corners is None:
        raise RuntimeError("Pitch not detected in the first frame")
    pitch_corners = detected_corners.reshape(-1, 1, 2).astype(np.float32)

    lk_params = dict(
        winSize=(21, 21),
        maxLevel=3,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)
    )
    prev_gray = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)
    post_tracker = PostTracker()

    goals_info: List[Tuple[int, Optional[Tuple[Tuple[int, int], Tuple[int, int]]]]] = []
    frame_index = 0
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        pitch_corners, current_gray = update_pitch_corners(frame, pitch_corners, prev_gray, frame_index, lk_params)

        goal_coordinates = detect_goal_in_frame(frame, pitch_corners, post_tracker)
        goals_info.append((frame_index, goal_coordinates))

        draw_black_goal(frame)
        overlay_pitch_info(frame, pitch_corners)

        cv2.imshow("goals", frame)
        if writer:
            writer.write(frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Exit if 'Esc' is pressed
            break

        prev_gray = current_gray
        frame_index += 1

    cap.release()
    if writer:
        writer.release()
    cv2.destroyAllWindows()

    elapsed_time = time.time() - start_time
    fps = frame_index / elapsed_time if elapsed_time > 0 else 0
    print(f"[DONE] {frame_index} frames processed at {fps:.1f} FPS")
    return goals_info

video_goals = process_video(Path("data/Match.mp4"), Path("data/goals.mp4"))
for idx, coords in video_goals:
    print(f"Frame {idx}: {coords}")

## Ball tracking using **YOLO**

In [ ]:
model = YOLO("weights/best.pt")


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply perspective transformation
    warped = cv2.warpPerspective(frame, H, (1080, 1920))

    # Get tracking results from YOLO
    results = model.track(warped, persist=True, conf=0.5)[0]

    # Check if the model found an object
    if results and results.boxes is not None:
        for box in results.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Rectangle Drawing
            cv2.rectangle(warped, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(
                warped,
                f"Ball {conf:.2f}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 0),
                2,
            )

            # Center of the object
            center = ((x1 + x2) // 2, (y1 + y2) // 2)
            last_valid_positions.append(center)
            if len(last_valid_positions) > max_positions:
                last_valid_positions.pop(0)

            # Trajectory drawing
            for i in range(1, len(last_valid_positions)):
                cv2.line(
                    warped,
                    last_valid_positions[i - 1],
                    last_valid_positions[i],
                    (0, 0, 255),
                    2,
                )

            break  # Only first found object

    # Write the frame to the output
    out.write(warped)

    # # Показ
    scaled_warped = cv2.resize(warped, (0, 0), fx=0.4, fy=0.4)
    cv2.imshow("Warped View", scaled_warped)

    if cv2.waitKey(30) & 0xFF == 27:
        break

## Combined code

In [9]:
import time
from pathlib import Path
from dataclasses import dataclass
from typing import Final, Optional, Tuple, List

import cv2
import numpy as np
from ultralytics import YOLO

@dataclass(frozen=True)
class PitchConfig:
    green_range: Tuple[Tuple[int, int, int], Tuple[int, int, int]] = ((35, 40, 40), (85, 255, 255))
    redetection_interval: int = 30
    max_corner_displacement: float = 6.0

@dataclass(frozen=True)
class GoalPostConfig:
    strip_height: int = 80
    bright_hsv_range: Tuple[Tuple[int, int, int], Tuple[int, int, int]] = ((0, 0, 200), (180, 60, 255))
    sobel_blur_kernel: int = 21
    min_peak_intensity_ratio: float = 0.30
    min_peak_gap: int = 80
    zone_fraction: float = 0.4
    max_allowed_jump: int = 30
    recheck_radius: int = 3
    recheck_threshold: float = 0.40
    smoothing_alpha: float = 0.35

LABEL_FONT: Final = cv2.FONT_HERSHEY_SIMPLEX
PITCH_CONFIG = PitchConfig()
GOAL_CONFIG = GoalPostConfig()

def order_corners(points: np.ndarray) -> np.ndarray:
    """
    Orders the input points into [Top-Left, Top-Right, Bottom-Right, Bottom-Left].
    """
    s = points.sum(axis=1)
    d = np.diff(points, axis=1)
    ordered = np.array([
        points[np.argmin(s)],
        points[np.argmin(d)],
        points[np.argmax(s)],
        points[np.argmax(d)]
    ], dtype=np.float32)
    return ordered

def detect_pitch(hsv_frame: np.ndarray) -> Optional[np.ndarray]:
    """
    Detects the green pitch region in an HSV image.
    Returns a 4x2 array of the pitch corner points (ordered TL, TR, BR, BL) or None.
    """
    lower_green, upper_green = PITCH_CONFIG.green_range
    mask = cv2.inRange(hsv_frame, lower_green, upper_green)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    largest = max(contours, key=cv2.contourArea)
    if cv2.contourArea(largest) < 50_000:
        return None
    
    hull = cv2.convexHull(largest)
    epsilon = 0.02 * cv2.arcLength(largest, True)
    approx = cv2.approxPolyDP(hull, epsilon, True)
    if len(approx) != 4:
        return None

    return order_corners(approx.reshape(-1, 2))

def get_bright_mask(hsv_frame: np.ndarray) -> np.ndarray:
    """
    Returns a binary mask for pixels within the bright HSV range.
    """
    lower_bright, upper_bright = GOAL_CONFIG.bright_hsv_range
    return cv2.inRange(hsv_frame, lower_bright, upper_bright)

def detect_two_peaks(bgr_strip: np.ndarray) -> Optional[Tuple[int, int]]:
    """
    From a given vertical strip (in BGR) of the pitch, finds the x positions of two bright peaks (goal posts).
    Returns (left_peak, right_peak) or None if no valid peaks are found.
    """
    hsv_strip = cv2.cvtColor(bgr_strip, cv2.COLOR_BGR2HSV)
    mask = cv2.morphologyEx(
        get_bright_mask(hsv_strip),
        cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    )
    energy = np.abs(cv2.Sobel(mask, cv2.CV_32F, 0, 1, ksize=3)).sum(axis=0)
    energy = cv2.blur(energy[:, None], (GOAL_CONFIG.sobel_blur_kernel, 1)).ravel()
    if energy.max() < 1:
        return None

    total_width = energy.size
    left_zone = int(GOAL_CONFIG.zone_fraction * total_width)
    right_zone = int((1 - GOAL_CONFIG.zone_fraction) * total_width)
    left_peak = np.argmax(energy[:left_zone])
    right_peak = np.argmax(energy[right_zone:]) + right_zone

    threshold = GOAL_CONFIG.min_peak_intensity_ratio * energy.max()
    if (energy[left_peak] < threshold or 
        energy[right_peak] < threshold or 
        (right_peak - left_peak) < GOAL_CONFIG.min_peak_gap):
        return None

    return int(left_peak), int(right_peak)

def is_bright_confirmed(hsv_strip: np.ndarray, x: int) -> bool:
    """
    Checks if a narrow vertical slice around x in hsv_strip has enough bright pixels.
    """
    w = hsv_strip.shape[1]
    x0, x1 = max(0, x - GOAL_CONFIG.recheck_radius), min(w, x + GOAL_CONFIG.recheck_radius + 1)
    brightness_ratio = (get_bright_mask(hsv_strip[:, x0:x1]) > 0).mean()
    return brightness_ratio >= GOAL_CONFIG.recheck_threshold

class PostTracker:
    """
    Uses an exponential moving average to keep the goal post positions stable across frames.
    """
    def __init__(self) -> None:
        self.last_posts: Optional[Tuple[int, int]] = None

    def update(self, measurement: Optional[Tuple[int, int]], hsv_strip: np.ndarray) -> Optional[Tuple[int, int]]:
        if measurement:
            left, right = measurement
            if not (is_bright_confirmed(hsv_strip, left) and is_bright_confirmed(hsv_strip, right)):
                measurement = None

        if self.last_posts and measurement:
            if max(abs(measurement[0] - self.last_posts[0]),
                   abs(measurement[1] - self.last_posts[1])) > GOAL_CONFIG.max_allowed_jump:
                measurement = None

        if measurement:
            if self.last_posts:
                smoothed_left = int(GOAL_CONFIG.smoothing_alpha * measurement[0] +
                                    (1 - GOAL_CONFIG.smoothing_alpha) * self.last_posts[0])
                smoothed_right = int(GOAL_CONFIG.smoothing_alpha * measurement[1] +
                                     (1 - GOAL_CONFIG.smoothing_alpha) * self.last_posts[1])
                self.last_posts = (smoothed_left, smoothed_right)
            else:
                self.last_posts = measurement
        return self.last_posts

def draw_black_goal(frame: np.ndarray) -> Optional[Tuple[Tuple[int, int], Tuple[int, int]]]:
    """
    Detects and draws a fixed black-area (goal) on the given frame.
    Returns the top-left and bottom-right coordinates of the detected goal, or None if not found.
    """
    h, w = frame.shape[:2]
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    black_mask = cv2.inRange(hsv, (0, 0, 0), (180, 255, 40))
    black_mask = cv2.morphologyEx(black_mask, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8), iterations=2)
    cnts, _ = cv2.findContours(black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        x, y, bw, bh = cv2.boundingRect(cnt)
        y += 50
        if not ((w * 0.05) * (h * 0.015) < area < (w * 0.15) * (h * 0.045)):
            continue
        ar = bw / bh
        if not (2.5 < ar < 6.0) or not (y + bh / 2 < h * 0.25) or not (w * 0.3 < x + bw / 2 < w * 0.7):
            continue
        cv2.rectangle(frame, (x, y), (x + bw, y + bh), (255, 0, 255), 2)
        return (x, y), (x + bw, y + bh)
    return None

def overlay_pitch_info(frame: np.ndarray, corners: np.ndarray) -> None:
    """
    Overlays the pitch area (polygon and corner labels) onto the frame.
    """
    poly = corners.astype(int).reshape(-1, 2)
    cv2.polylines(frame, [poly], True, (0, 255, 0), 2)
    labels = ("TL", "TR", "BR", "BL")
    for label, (x, y) in zip(labels, poly):
        cv2.putText(frame, label, (x + 5, y - 5), LABEL_FONT, 0.5, (0, 255, 0), 1)

def update_pitch_corners(
    frame: np.ndarray,
    current_corners: np.ndarray,
    prev_gray: np.ndarray,
    frame_idx: int,
    lk_params: dict,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Either re-detects the pitch or updates its corners via optical flow.
    Returns the possibly updated corners and the current grayscale frame.
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if frame_idx % PITCH_CONFIG.redetection_interval == 0:
        new_detection = detect_pitch(hsv)
        if new_detection is not None:
            current_corners = new_detection.reshape(-1, 1, 2).astype(np.float32)
    current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    new_corners, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, current_gray, current_corners, None, **lk_params)
    if status.sum() == 4 and np.max(np.linalg.norm(new_corners - current_corners, axis=2)) < PITCH_CONFIG.max_corner_displacement:
        current_corners = new_corners
    return current_corners, current_gray

def detect_goal_in_frame(
    frame: np.ndarray, 
    pitch_corners: np.ndarray, 
    tracker: PostTracker
) -> Optional[Tuple[Tuple[int, int], Tuple[int, int]]]:
    """
    Using the lower part (strip) of the pitch, detects goal posts and returns a goal rectangle.
    """
    poly = pitch_corners.astype(int).reshape(-1, 2)
    xL, xR = poly[:, 0].min(), poly[:, 0].max()
    yT, yB = poly[:, 1].min(), poly[:, 1].max()

    strip_top = max(int(yB - GOAL_CONFIG.strip_height), 0)
    strip_bottom = int(yB)
    strip_bgr = frame[strip_top:strip_bottom, xL:xR]
    strip_hsv = cv2.cvtColor(strip_bgr, cv2.COLOR_BGR2HSV)

    measurement = detect_two_peaks(strip_bgr)
    posts = tracker.update(measurement, strip_hsv)

    if posts:
        left_post, right_post = posts
        goal_tl = (xL + left_post, strip_top+20)
        goal_br = (xL + right_post, yB)
        cv2.rectangle(frame, goal_tl, goal_br, (255, 0, 255), 2)
        return goal_tl, goal_br
    return None

def process_video_with_yolo_and_goals(source_path: Path, dest_path: Optional[Path] = None) -> None:
    cap = cv2.VideoCapture(str(source_path))
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Cannot read video")
    height, width = frame.shape[:2]
    
    warp_dims = (1080, 1920)
    
    writer = None
    if dest_path:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(str(dest_path), fourcc, 30, warp_dims)
    
    ball_model = YOLO("weights/best.pt")
    
    H = np.eye(3, dtype=np.float32)
    
    last_valid_positions: List[Tuple[int, int]] = []
    max_positions = 50

    warped = cv2.warpPerspective(frame, H, warp_dims)
    initial_hsv = cv2.cvtColor(warped, cv2.COLOR_BGR2HSV)
    init_corners = detect_pitch(initial_hsv)
    if init_corners is None:
        raise RuntimeError("Pitch not detected in first frame")
    pitch_corners = init_corners.reshape(-1, 1, 2).astype(np.float32)

    lk_params = dict(
        winSize=(21, 21),
        maxLevel=3,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)
    )
    prev_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    
    post_tracker = PostTracker()
    
    cv2.namedWindow("Detection", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Detection", 960, 540)
    
    frame_idx = 0
    start_time = time.time()
    goal_cooldown = 0  # Prevents double-scoring
    player1_score = 0  # Top player
    player2_score = 0  # Bottom player
    ball_inside_bottom_goal = False
    ball_inside_upper_goal = False
    missing_counter = 0
    max_missing_frames = 5 
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        warped = cv2.warpPerspective(frame, H, warp_dims)

     

        # ---- Pitch & Goal-Post Detection ----
        pitch_corners, current_gray = update_pitch_corners(warped, pitch_corners, prev_gray, frame_idx, lk_params)
        bottom_gate_coords = detect_goal_in_frame(warped, pitch_corners, post_tracker)
        if bottom_gate_coords:
            cv2.putText(warped, "Goal Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
  
        upper_gate_coords = draw_black_goal(warped)
        overlay_pitch_info(warped, pitch_corners)
  
        # ---- YOLO Ball Detection ----
        results = ball_model.track(
            warped, 
            persist=True, 
            conf=0.3,  # Lower this if needed
            tracker="bytetrack.yaml",  # or "bytetrack.yaml"
            imgsz=640,  # Match your model's training size
            iou=0.3,  # Adjust intersection over union threshold
            verbose=False
        )[0]
        if results and results.boxes is not None:
            for box in results.boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
  
                # Draw detected ball rectangle and confidence label.
                cv2.rectangle(warped, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(warped, f"Ball {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
  
                # Compute center and update trajectory.
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                last_valid_positions.append(center)
                if len(last_valid_positions) > max_positions:
                    last_valid_positions.pop(0)
  
                # # Check if ball is in bottom gate (goal for Player 2)
                # if goal_cooldown == 0 and bottom_gate_coords:
                #     gate_tl, gate_br = bottom_gate_coords
                #     if gate_tl[0] <= center[0] <= gate_br[0] and gate_tl[1] <= center[1] <= gate_br[1]:
                #         player2_score += 1
                #         print(f"GOAL for Player 2! Score: {player1_score} - {player2_score}")
                        
                # # Check if ball is in upper gate (goal for Player 1)
                # if goal_cooldown == 0 and upper_gate_coords:
                #     upper_gate_tl, upper_gate_br = upper_gate_coords
                #     if upper_gate_tl[0] <= center[0] <= upper_gate_br[0] and upper_gate_tl[1] <= center[1] <= upper_gate_br[1]:
                #         player1_score += 1
                #         print(f"GOAL for Player 1! Score: {player1_score} - {player2_score}")

                # Draw trajectory lines.
                for i in range(1, len(last_valid_positions)):
                    cv2.line(warped, last_valid_positions[i - 1], last_valid_positions[i], (0, 0, 255), 2)
                break  # Only process the first detected ball

        if goal_cooldown > 0:
            goal_cooldown -= 1

        if results and results.boxes is not None:
            missing_counter = 0
            cx, cy = last_valid_positions[-1]

            if bottom_gate_coords:
                x1, y1 = bottom_gate_coords[0]
                x2, y2 = bottom_gate_coords[1]
                if x1 < cx < x2 and y1 < cy < y2:
                    ball_inside_bottom_goal = True

            if upper_gate_coords:
                x1, y1 = upper_gate_coords[0]
                x2, y2 = upper_gate_coords[1]
                if x1 < cx < x2 and y1 < cy < y2:
                    ball_inside_upper_goal = True
        else:
            missing_counter += 1
            if missing_counter > max_missing_frames and goal_cooldown == 0:
                if ball_inside_bottom_goal:
                    player1_score += 1
                    print("⚽ Goal for Player 1!")
                    goal_cooldown = 10 
                if ball_inside_upper_goal:
                    player2_score += 1
                    print("⚽ Goal for Player 2!")
                    goal_cooldown = 10

                ball_inside_bottom_goal = False
                ball_inside_upper_goal = False
                missing_counter = 0

        score_text = f"Player 1 - {player1_score} | Player 2 - {player2_score}"
        text_size, _ = cv2.getTextSize(score_text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)
        text_x = (warped.shape[1] - text_size[0]) // 2
        text_y = warped.shape[0] - 30
        cv2.putText(warped, score_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)

        cv2.imshow("Detection", warped)
        if writer:
            writer.write(warped)
        if cv2.waitKey(1) & 0xFF == 27:
            break
  
        prev_gray = current_gray
        frame_idx += 1
  
    cap.release()
    if writer:
        writer.release()
    cv2.destroyAllWindows()
    elapsed = time.time() - start_time
    fps = frame_idx / elapsed if elapsed > 0 else 0
    print(f"[DONE] {frame_idx} frames processed at {fps:.1f} FPS")

process_video_with_yolo_and_goals(Path("data/Match.mp4"), Path("data/combined_output.mp4"))

⚽ Goal for Player 2!
⚽ Goal for Player 2!
⚽ Goal for Player 2!
⚽ Goal for Player 1!
⚽ Goal for Player 2!
⚽ Goal for Player 2!
⚽ Goal for Player 2!
⚽ Goal for Player 2!
⚽ Goal for Player 1!
⚽ Goal for Player 1!
⚽ Goal for Player 1!
⚽ Goal for Player 1!
⚽ Goal for Player 1!
⚽ Goal for Player 1!
⚽ Goal for Player 2!
⚽ Goal for Player 1!
⚽ Goal for Player 2!
⚽ Goal for Player 2!
[DONE] 10327 frames processed at 14.4 FPS
